# Introduzione

L'environment **anaconda** da utilizzare è **ddmcs**.  

# Importazione pacchetti

Importare i vari pacchetti necessari  per eseguire le azioni elencate nell'introduzione:
   - selenium (per navigare sulle pagine web)
   - request (scaricare dati da internet)
   - bs4 (beautiful soup, manipolare gli html)
   - os (navigare nei file del computer)
   - pandas (manipolazione/esportazione dataframes)
   - time (per i time sleep)
   - datetime (gestire i timestamp, ovvero le stringhe contenti date e orari)
   - tqdm (per le barre di progresso) 
   - re (lavorare con regex e le regular expressions)

In [100]:
#%% Pacchetti
## Importazione pacchetti per selenium
# Altre funzionalità di selenium
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.common.action_chains import ActionChains
#from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
#from selenium.common.exceptions import TimeoutException
#from webdriver_manager.chrome import ChromeDriverManager
#-------##-------##-------##-------##-------##-------##-------##-------##-------##-------#
import requests # Importazione pacchetti per scaricare dati da internet
from bs4 import BeautifulSoup # Importazione pacchetti per gestire HTML
import os # Interagire con i file del pc 
from tqdm import tqdm as tqdm # Importazione pacchetti per barra di progresso
import re # Importazione pacchetti per utilizzare regex e le regular expression
import pandas as pd  # Importazione pacchetti per lavorare con i dataframe
import warnings # Importazione pacchetti per gestire i warnings, ad esempio quelli di pandas
from datetime import datetime# Importazione pacchetti per lavorare con timestamps, orari e date
import numpy as np 

#pd.set_option('display.max_rows', None)

In [101]:
# Directory principale dello scraper, ove vi sono gli script e gli html devono essere posizionati
path_main_directory = '/Users/mattia/Desktop/Università/Data Science in Python/14) Web Scraping/Scraping from booking'

In [103]:
# Importare l'html del file in questione
with open(f"{path_main_directory}/html/html_final.txt","r") as f:
   html = f.read()

html_bs4 = BeautifulSoup(html,'html.parser')
proprietà = html_bs4.find_all("div",{"data-testid":"property-card"})

booking_dataframe = pd.DataFrame(columns=[
    "titolo",
    "recensione_voto_numerico",
    "recensione_voto_parola",
    "numero_recensioni",
    "permanenza",
    "prezzo",
    "distanza_centro",
    "indirizzo",
    "stelle",
    "valutazione_booking",
    "descrizione_camera",
    "configurazione_camera",
    "descrizione_unprocessed"
])


for post in tqdm(proprietà):
   # Prendere il titolo della proprietà
   titolo = post.find("div",{"data-testid":"title"}).text
   
   # Selezionare la tag delle recensioni
   review = post.find("div",{"data-testid":"review-score"})#.text
   
   ## In caso di nuovi insererimenti  (novità su booking) non ci sono votazioni.
   if review != None:
      ## Estrarre voto numerico dalla tag delle recensioni
      recensione_voto_numerico = review.find("div",{"aria-hidden":"true"}).text

      ## Estrarre il voto in parole ed il numero di recensioni dalla tag delle recensioni
      info_recensioni = review.find("div",{"aria-hidden":"false"})
      for div in info_recensioni:
         if ("recensioni" in div.text.lower()) or ("recensione" in div.text.lower()):
            numero_recensioni = div.text

         else:
            recensione_voto_parola = div.text
   else:
      recensione_voto_numerico = "Novità su booking"
      numero_recensioni = "Novità su booking"
      recensione_voto_parola = "Novità su booking"
         

   # Selezionare la tag dei prezzi e per il numero di persone e notti
   info_costi = post.find("div",{"data-testid":"availability-rate-wrapper"})#.text

   ## Prezzo per notti e numero di persone
   permanenza =  info_costi.find("div",{"data-testid":"price-for-x-nights"}).text
   
   ## Prezzo
   prezzo =  info_costi.find("span",{"data-testid":"price-and-discounted-price"}).text


   # Selezionare la località
   ## Distanza dal centro
   distanza_centro = post.find("span",{"data-testid":"distance"}).text


   indirizzo = post.find("span",{"data-testid":"address"}).text


   # Valutazioni

   ## Rating stars
   rating_stars = post.find("div",{"data-testid":"rating-stars"})
   
   ### La proprietà potrebbe non essere un albergo e quindi non avere stelle
   if rating_stars != None:
      stelle = len(rating_stars.find_all("div"))
   else:
      stelle = np.nan
         
   ## Rating booking
   rating_booking = post.find("div",{"data-testid":"rating-squares"})
   
   ### La proprietà potrebbe  essere un albergo e quindi non avere valutazione booking
   if rating_booking != None:
      valutazione_booking = len(rating_booking.find_all("div"))
   else:
      valutazione_booking = np.nan


   # Descrizione della camera offerta
   
   ## Tutta la descrizione
   descrizione_unprocessed = post.find("div",{"data-testid":"recommended-units"}).text
   
   ## tag descrizione per estrarre cose estraibili
   descrizione = post.find("div",{"data-testid":"recommended-units"})

   ### Descrizione camera
   descrizione_camera = descrizione.find("h4").text
   
   ### Configurazione camera, che potrebbe non esistere 
   configurazione_camera = descrizione.find("div",{"data-testid":"property-card-unit-configuration"})

   if  configurazione_camera != None:
      configurazione_camera = configurazione_camera.text

   else:
      configurazione_camera = "Configurazione camera non disponibile"



   # Crea il dizionario con i dati della singola proprietà
   informazioni_singola_proprietà = {  "titolo": titolo,
                                       "recensione_voto_numerico": recensione_voto_numerico,
                                       "recensione_voto_parola": recensione_voto_parola,
                                       "numero_recensioni": numero_recensioni,
                                       "permanenza": permanenza,
                                       "prezzo": prezzo,
                                       "distanza_centro": distanza_centro,
                                       "indirizzo": indirizzo,
                                       "stelle": stelle,
                                       "valutazione_booking": valutazione_booking,
                                       "descrizione_camera": descrizione_camera,
                                       "configurazione_camera": configurazione_camera,
                                       "descrizione_unprocessed": descrizione_unprocessed

                                    }
   
   # Aggiunge direttamente come nuova riga
   booking_dataframe.loc[len(booking_dataframe)] = informazioni_singola_proprietà

100%|██████████| 67/67 [00:00<00:00, 599.05it/s]
